# data
This paper incorporates a diverse range of datasets related to properties across Scotland. One of the main contributions of our work is acquiring and merging these datasets into a single dataset. We detail each dataset.

In [1]:
import pandas as pd
import openpyxl
import xlrd
import pyxlsb
import folium
from datetime import date
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [3]:
import warnings
import time

warnings.simplefilter('ignore', DeprecationWarning)
time.clock()   #禁止导入用时警告

31.0002581

Since we need to visualise some aspects of these datasets geographically on a map.

In [17]:
import sys
!{sys.executable} -m pip install folium

In [2]:
# print(pd.show_versions())#查看版本
#!python -m pip install --upgrade pip  #update pip
#!pip install pyxlsb   #update pyxlsb
#!pip install pandas

# SW - All Lead WQ Samples (2010-18)
Water quality data (including lead concentration values) collected from houoses in Scotland from 2010 to 2018 using various sampling procedures.

In [53]:
sw_all_lead = pd.read_excel('SW - All Lead WQ Samples (2010-18).xls', sheet_name='Report1', index=False, encoding='utf8')
sw_all_lead.head()

,Sample Date Timestamp,Result Numeric Entry,DMA Name,DMA Id,RSZ Name,RSZ Id,RSZ Water System Id,RSZ Water System Name,Sample Point Matrix,Det Test Description,...,Street Postcode,District Postcode,Eastings,Northings,Region Name 2006,WOA Name,WOA Id,WSZ Name,WSZ Id,Site Name
0,2011-12-01 13:30:00,42.0,Abbey Drive DMA,DMA001829,Milngavie M4c,RSZ004060,WTW000182,MILNGAVIE WTW 2007 NS562760,POT_WATER,Lead,...,G14 9LY,G14,254269.0,667584.0,CLYDE,Milngavie & Balmore WOA,WOA000402,M4 Mugdock C WSZ,WSZ001641,UNKNOWN
1,2012-01-23 12:00:00,0.2,Abbey Drive DMA,DMA001829,Milngavie M4c,RSZ004060,WTW000182,MILNGAVIE WTW 2007 NS562760,POT_WATER,Lead,...,G14 9PD,G14,253988.0,667584.0,CLYDE,Milngavie & Balmore WOA,WOA000402,M4 Mugdock C WSZ,WSZ001641,UNKNOWN
2,2012-01-26 13:15:00,0.8,Abbey Drive DMA,DMA001829,Milngavie M4c,RSZ004060,WTW000182,MILNGAVIE WTW 2007 NS562760,POT_WATER,Lead,...,G14 9JJ,G14,254414.0,667890.0,CLYDE,Milngavie & Balmore WOA,WOA000402,M4 Mugdock C WSZ,WSZ001641,UNKNOWN
3,2013-04-13 11:10:53,0.4,Abbey Drive DMA,DMA001829,Milngavie M4c,RSZ004060,WTW000182,MILNGAVIE WTW 2007 NS562760,POT_WATER,Lead,...,G14 9LZ,G14,254248.0,667512.0,CLYDE,Milngavie & Balmore WOA,WOA000402,M4 Mugdock C WSZ,WSZ001641,UNKNOWN
4,2013-04-16 10:45:00,0.2,Abbey Drive DMA,DMA001829,Milngavie M4c,RSZ004060,WTW000182,MILNGAVIE WTW 2007 NS562760,POT_WATER,Lead,...,G14 9LY,G14,254238.0,667605.0,CLYDE,Milngavie & Balmore WOA,WOA000402,M4 Mugdock C WSZ,WSZ001641,MILNGAVIE M4C RSZ


In [54]:
#Firstly, we delete those cancelled samples which have no lead concentration result.
col_result = sw_all_lead.loc[:,['Result Numeric Entry']]  
col_result = col_result[col_result['Result Numeric Entry'].isin(['NaN'])]
na_index = col_result.index  # get the indexs of those NA rows
print(len(na_index))

sw_all_lead=sw_all_lead.drop(sw_all_lead.index[na_index])#  delete the rows with this NA index  
sw_all_lead=sw_all_lead.reset_index()    # reset a new index for the new dataframe
sw_all_lead=sw_all_lead.drop(columns=['index'])   

1232


After dropping 1232 useless samples from the dataset, we can do the following analysis:

In [56]:
sw_all_lead.loc[:,['Result Numeric Entry']] .describe()

,Result Numeric Entry
count,64300.000000
mean,16.234974
std,467.416708
min,0.200000
25%,0.200000
50%,0.500000
75%,4.200000
max,70948.600000


In [58]:
# 删除了lead result 的缺失值之后，发现其他变量也有很多缺失值
sw_all_lead.loc[sw_all_lead['Result Numeric Entry']>10,:]  #>10的有很多都是啥数据都没有，没有邮编，但有的>10 的有邮编

,Sample Date Timestamp,Result Numeric Entry,DMA Name,DMA Id,RSZ Name,RSZ Id,RSZ Water System Id,RSZ Water System Name,Sample Point Matrix,Det Test Description,...,Street Postcode,District Postcode,Eastings,Northings,Region Name 2006,WOA Name,WOA Id,WSZ Name,WSZ Id,Site Name
0,2011-12-01 13:30:00,42.0,Abbey Drive DMA,DMA001829,Milngavie M4c,RSZ004060,WTW000182,MILNGAVIE WTW 2007 NS562760,POT_WATER,Lead,...,G14 9LY,G14,254269.0,667584.0,CLYDE,Milngavie & Balmore WOA,WOA000402,M4 Mugdock C WSZ,WSZ001641,UNKNOWN
15,2011-03-18 21:24:04,21.6,Abbeylands Road DMA,DMA001418,Burncrooks,RSZ004022,WTW000033,BURNCROOKS WTW 1960 NS502796,POT_WATER,Lead,...,G81 6QY,G81,249956.0,673015.0,FORTH,Burncrooks WOA,WOA000075,Burncrooks 21 WSZ,WSZ000259,BURNCROOKS RSZ
17,2011-03-19 12:10:00,172.3,Abbeylands Road DMA,DMA001418,Burncrooks,RSZ004022,WTW000033,BURNCROOKS WTW 1960 NS502796,POT_WATER,Lead,...,G81 5JJ,G81,249908.0,673200.0,FORTH,Burncrooks WOA,WOA000075,Burncrooks 21 WSZ,WSZ000259,BURNCROOKS RSZ
18,2011-03-21 10:30:00,16.3,Abbeylands Road DMA,DMA001418,Burncrooks,RSZ004022,WTW000033,BURNCROOKS WTW 1960 NS502796,POT_WATER,Lead,...,G81 5PD,G81,250310.0,672661.0,CLYDE,Burncrooks WOA,WOA000075,Burncrooks 21 WSZ,WSZ000259,BURNCROOKS WTW 1960 NS502796
19,2011-03-21 10:40:00,16.2,Abbeylands Road DMA,DMA001418,Burncrooks,RSZ004022,WTW000033,BURNCROOKS WTW 1960 NS502796,POT_WATER,Lead,...,G81 5PD,G81,250286.0,672667.0,CLYDE,Burncrooks WOA,WOA000075,Burncrooks 21 WSZ,WSZ000259,BURNCROOKS WTW 1960 NS502796
28,2014-12-19 10:30:00,14.9,Abbeylands Road DMA,DMA001418,Burncrooks,RSZ004022,WTW000033,BURNCROOKS WTW 1960 NS502796,POT_WATER,Lead,...,G81 5PD,G81,250308.0,672665.0,FORTH,Burncrooks WOA,WOA000075,Burncrooks 21 WSZ,WSZ000259,UNKNOWN
29,2015-01-23 08:00:00,21.3,Abbeylands Road DMA,DMA001418,Burncrooks,RSZ004022,WTW000033,BURNCROOKS WTW 1960 NS502796,POT_WATER,Lead,...,G81 5PD,G81,250308.0,672665.0,FORTH,Burncrooks WOA,WOA000075,Burncrooks 21 WSZ,WSZ000259,UNKNOWN
33,2015-10-28 10:05:00,91.8,Abbeylands Road DMA,DMA001418,Burncrooks,RSZ004022,WTW000033,BURNCROOKS WTW 1960 NS502796,POT_WATER,Lead,...,G81 5LJ,G81,250159.0,672933.0,FORTH,Burncrooks WOA,WOA000075,Burncrooks 21 WSZ,WSZ000259,UNKNOWN
34,2015-11-03 09:05:00,11.3,Abbeylands Road DMA,DMA001418,Burncrooks,RSZ004022,WTW000033,BURNCROOKS WTW 1960 NS502796,POT_WATER,Lead,...,G81 5LJ,G81,250159.0,672933.0,FORTH,Burncrooks WOA,WOA000075,Burncrooks 21 WSZ,WSZ000259,UNKNOWN
170,2011-04-20 10:00:00,15.1,Aberdour DMA,DMA004295,Glendevon B,RSZ001015,WTW000636,GLENFARG WTW,POT_WATER,Lead,...,KY3 0UE,KY3,319289.0,685572.0,TAY,Glendevon WOA,WOA000170,Aberdour WSZ,WSZ001753,UNKNOWN


In [19]:
m = folium.Map(
    location=[55.924550, -3.176920],
    zoom_start=15
)

In [ ]:
def location(latitude,longitude,m,start_name,size):
    '''
    This function is  mark house location on the map by a red circle.
    The cycle has radius=0.0025 * size. Size is the times of the station has been used 
    '''
    folium.CircleMarker(
    location=[latitude, longitude],
    radius=0.1*size,
    color='red',
    fill=True,
    fill_color='red'
    ).add_to(m)
    folium.Marker(
    location=[latitude, longitude],
    popup=start_name,
    icon=folium.Icon(color='green')
    ).add_to(m)
    return m

In [5]:
sw_comm_pipe = pd.read_excel('SW - Comm pipe data.xls', sheet_name='Survey Results', index=False, encoding='utf8')
sw_comm_pipe.head()

,AR10_PROPERTYID,EASTING_NUM,NORTHING_NUM,AR10_MATERIAL,AR10_COMMS,MAIN_DISTANCE,FULL_ADDRESS,POST_TOWN,Street postcode,DISTRICT POSTCODE,...,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58
0,2.464474e+17,246447400.0,672980100.0,MDPE,1,9.934,"7, KILPATRICK COURT, OLD KILPATRICK, GLASGOW, ...",GLASGOW,G605JA,G60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.467539e+17,246753900.0,672904800.0,LEAD,1,17.914,"BALCLUTHA, STATION ROAD, OLD KILPATRICK, GLASG...",GLASGOW,G605LT,G60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.477474e+17,247747400.0,671522100.0,LEAD,1,12.712,"7, WELLINGTON PLACE, CLYDEBANK, G81 4JN",CLYDEBANK,G814JN,G81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.477931e+17,247793100.0,671514100.0,LEAD,1,25.427,"16, WELLINGTON PLACE, CLYDEBANK, G81 4JN",CLYDEBANK,G814JN,G81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.475032e+17,247503200.0,671822500.0,MDPE,1,18.539,"25, DELHI AVENUE, CLYDEBANK, G81 4JS",CLYDEBANK,G814JS,G81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
sw_comm_pipe.columns # 有很多空列：unnamed  --->.drop

Index(['AR10_PROPERTYID', 'EASTING_NUM', 'NORTHING_NUM', 'AR10_MATERIAL',
       'AR10_COMMS', 'MAIN_DISTANCE', 'FULL_ADDRESS', 'POST_TOWN',
       'Street postcode', 'DISTRICT POSTCODE', 'RZ_REF', 'RZ_NAME', 'REGION',
       'CONFIDENCE_GRADE', 'Date_Commissioned', 'AGE',
       'Pipe age in years from 2019', 'OSAPR', 'WTW1', 'WTW2', 'WTW3',
       'Survey this toby?', 'Date', 'Inspected by', 'Easting', 'Northing',
       'Description of Location', 'Domestic or Non-domestic', 'Property Type',
       'If Other', 'Toby cover photo reference',
       'How many properties are supplied by this comms pipe',
       'toby box acessed?', 'Address, if different ', 'Is there a meter box?',
       'How certain is  the identification?',
       'Is there any evidence of leakage?', 'Is the property age pre 1970?',
       'Pipe Material ', 'Specify If Other ', 'Nearside or Farside?',
       'Estimated length of pipe in meters', 'Comments',
       'Material type changed', 'Incompete record', 'Unnamed:

In [7]:
sw_lead_replace=pd.read_csv("SW - Lead Comm Pipe Replacements (2004-2018).csv")
sw_lead_replace.head()

,Work Order Id,Std Job No,WO Completed Status,WO Create Date,Ads Building Number,Ads Eastings,Ads Northings,Ads Operation Area,Ads Osapr,Street postcode,Ads Post Town,Ads Region,Ads Water Operational Area,Ads Water Supply Zone
0,17598,WRNW17,All Complete ...,08/04/2004,13,"211,416.00","774,804.00",North West,APHTGJ8X5X1540903G,PH336NS,FORT WILLIAM,North,Camisky Wellfield WOA,Cowhill WSZ
1,17613,WRNW17,All Complete with Comments ...,08/04/2004,NaN,"345,995.00","644,362.00",South East,APJ14X8J5HJ5M5A0MT,TD12QT,GALASHIELS,South,Manse Street Galashiels WOA,Stow WSZ
2,17631,WRNW17,All Complete ...,08/04/2004,11,"341,551.00","727,292.00",North East,APXE0F875RC4DTU05J,DD68HL,NEWPORT-ON-TAY,East,LHills & GFarg & Glendevon WOA,Newport WSZ
3,21435,WRNW17,All Complete ...,09/04/2004,119,"262,575.00","660,849.00",South West,APAQD18G5JF4W6QGW3,G733PH,GLASGOW,West,Milngavie WOA,C2 Craigmaddie WSZ
4,21934,WRNW17,All Complete ...,12/04/2004,1,"336,254.00","950,204.00",North West,APQUAQ825EJ5QYE0DT,KW15NN,WICK,North,Loch Calder WOA,Toftcarl WSZ


In [8]:
sw_phosphate = pd.read_excel('SW - Phosphate Dosing WTWs Y or N.xlsx', sheet_name='Phosphate Dosed Site', skiprows=2, index=False, encoding='utf8')
sw_phosphate.head()

,ALEXANDRIA WTW 2000 NS383801,Yes,ALEXANDRIA WTW 2000 NS383801.1
0,AMLAIRD WTW NS483445,Yes,AMLAIRD WTW NS483445
1,ARDRISHAIG WTW 1993 NR846864,Yes,ARDRISHAIG WTW 1993 NR846864
2,ASSYNT WTW 2010 NH584667,Yes,ASSYNT WTW 2010 NH584667
3,AUCHNEEL WTW 2002 NX036618,Yes,AUCHNEEL WTW 2002 NX036618
4,AVIEMORE WTW 2011 NH891110,Yes,AVIEMORE WTW 2011 NH891110


In [9]:
#sw_postcode=pd.read_excel("SW - Postcodes linked to SW Zonal Structure.xlsb", engine = 'pyxlsb')
from pyxlsb import open_workbook

df=[]
with open_workbook('SW - Postcodes linked to SW Zonal Structure.xlsb') as wb:
    with wb.get_sheet(2) as sheet:
        for row in sheet.rows():
            df.append([item.v for item in row])

sw_postcode = pd.DataFrame(df[1:], columns=df[0])
sw_postcode.head()

,Street Postcode,District Postcode,WOA_ID,WOA_Name,RSZ_ID,RSZ_Name,WSZ_ID,WSZ_Name,DMA_ID,DMA_Name,Virtual_DMA_ID,Virtual_DMA_Name,None,None,Count,Current DMA?,WOA/RSZ/WSZ?,Total properties
0,AB101AA,AB10,WOA000199,Invercannie & Mannofield WOA,RSZ001029,Mannofield East,WSZ001061,Mannofield WSZ,DMA006865,George Street & King Street DMA,DMA006865,George Street & King Street DMA,None,None,,,,3020.0
1,AB101AB,AB10,WOA000199,Invercannie & Mannofield WOA,RSZ001031,Mannofield South,WSZ000037,Airyhall WSZ,DMA004023,Whitehall DMA (DMA004023),DMA004023,Whitehall DMA (DMA004023),None,None,,,,2440.0
2,AB101AF,AB10,WOA000199,Invercannie & Mannofield WOA,RSZ001029,Mannofield East,WSZ001061,Mannofield WSZ,DMA006865,George Street & King Street DMA,DMA006865,George Street & King Street DMA,None,None,,,,3020.0
3,AB101AG,AB10,WOA000199,Invercannie & Mannofield WOA,RSZ001029,Mannofield East,WSZ001061,Mannofield WSZ,DMA006865,George Street & King Street DMA,DMA006865,George Street & King Street DMA,None,None,,,,3020.0
4,AB101AH,AB10,WOA000199,Invercannie & Mannofield WOA,RSZ001031,Mannofield South,WSZ000037,Airyhall WSZ,DMA004023,Whitehall DMA (DMA004023),DMA004023,Whitehall DMA (DMA004023),None,None,,,,2440.0


In [10]:
sw_SWZ_phosphate = pd.read_excel('SW - Scottish Water Zonal Phosphate Levels.xls', sheet_name='Sheet1', skiprows=2, index=False, encoding='utf8')
sw_SWZ_phosphate.head()

,2018-08-08 00:00:00,11946116,Unnamed: 2,7.6,2.5,424,18.900000000000002,Ardrishaig Zone (Kilmartin STW),North
0,2018-08-15,11968896,NaN,7.5,2.3,431.0,15.8,Ardrishaig Zone (Kilmartin STW),North
1,2018-08-20,11977571,NaN,7.7,2.6,424.0,18.5,Ardrishaig Zone (Kilmartin STW),North
2,2018-08-29,12007845,NaN,7.6,2.3,435.0,19.1,Ardrishaig Zone (Kilmartin STW),North
3,2018-09-05,12036999,NaN,7.6,2.4,429.0,19.4,Ardrishaig Zone (Kilmartin STW),North
4,2018-09-12,12059686,NaN,7.7,2.5,422.0,19.3,Ardrishaig Zone (Kilmartin STW),North


In [11]:
workbook = xlrd.open_workbook('SW - All Lead WQ Samples (2010-18).xls')
worksheet=workbook.sheet_by_index(0)

booksheet = workbook.sheet_names()

print(workbook.sheet_names())                  #查看所有sheet
booksheet = workbook.sheet_by_index(0)         #用索引取第一个sheet
booksheet = workbook.sheet_by_name('Report1')  #或用名称取sheet
#读单元格数据
cell_11 = booksheet.cell_value(0,0)
cell_21 = booksheet.cell_value(1,0)
#读一行数据
row_3 = booksheet.row_values(2)
print(cell_11, cell_21, row_3)

['Report1']
Sample Date Timestamp 40878.5625 [40931.5, 0.2, 'Abbey Drive DMA', 'DMA001829', 'Milngavie M4c', 'RSZ004060', 'WTW000182', 'MILNGAVIE WTW 2007 NS562760', 'POT_WATER', 'Lead', 'µgPb/l', 40931.0, 'Jan', 2012.0, 'Authorised', 'C_ENQ_COMP', 'G14  9PD', 'G14', 253988.0, 667584.0, 'CLYDE', 'Milngavie & Balmore WOA', 'WOA000402', 'M4 Mugdock C WSZ', 'WSZ001641', 'UNKNOWN']


In [12]:
wb = openpyxl.load_workbook('SW - Phosphate Dosing WTWs Y or N.xlsx')
sheets = wb.sheetnames         #从名称获取sheet
booksheet = wb[sheets[0]]
 
rows = booksheet.rows
columns = booksheet.columns
#迭代所有的行
for row in rows:
    line = [col.value for col in row]

In [13]:
o_house_price=pd.read_csv('Other - UK-HPI-full-file-2019-03.csv')
o_house_price.head()

,Date,Year2,RegionName,CouncilArea2018Code,AveragePrice,Index,IndexSA,1m%Change,12m%Change,AveragePriceSA,...,NewPrice,NewIndex,New1m%Change,New12m%Change,NewSalesVolume,OldPrice,OldIndex,Old1m%Change,Old12m%Change,OldSalesVolume
0,01/01/2004,2004,Aberdeenshire,S12000034,81693.66964,40.864214,NaN,NaN,NaN,NaN,...,88436.13953,40.267250,NaN,NaN,103.0,81043.95084,40.883367,NaN,NaN,285.0
1,01/02/2004,2004,Aberdeenshire,S12000034,81678.76231,40.856757,NaN,-0.018248,NaN,NaN,...,88606.44649,40.344795,0.192576,NaN,107.0,80965.29542,40.843688,-0.097053,NaN,219.0
2,01/03/2004,2004,Aberdeenshire,S12000034,83525.09702,41.780317,NaN,2.260483,NaN,NaN,...,90296.91375,41.114508,1.907838,NaN,140.0,82903.23948,41.821302,2.393549,NaN,313.0
3,01/04/2004,2004,Aberdeenshire,S12000034,84333.67900,42.184780,NaN,0.968071,NaN,NaN,...,90319.87844,41.124964,0.025432,NaN,180.0,84003.99161,42.376586,1.327755,NaN,391.0
4,01/05/2004,2004,Aberdeenshire,S12000034,86379.95396,43.208353,NaN,2.426403,NaN,NaN,...,91989.17763,41.885039,1.848208,NaN,167.0,86222.73484,43.495852,2.641235,NaN,335.0


In [14]:
o_postcode=pd.read_csv("Other - Postcode_ household count_ urban class.csv", dtype='unicode')  # 'NationalPark2010Code'列type有问题
o_postcode.head()   

,Street postcode,District postcode,Sector postcode,DateOfIntroduction,DateOfDeletion,YearOfDeletion,GridReferenceEasting,GridReferenceNorthing,Latitude,Longitude,...,RegistrationDistrict2007Code,ROACommunityPlanningPartnership2006Code,ROALocal2006Code,StrategicDevelopmentPlanningArea2013Code,TravelToWorkArea2011Code,UrbanRural6Fold2016Code,UrbanRural8Fold2016Code,GridLinkIndicator,GridLinkPositionalAccuracy,NeverDigitised
0,AB452NL,AB45,AB45 2,01/08/1990 00:00,ND,ND,365169,865455,57.67760523,-2.58569683,...,S12000034,NaN,NaN,S11000001,S22000081,6,7,Y,1,NaN
1,AB452NN,AB45,AB45 2,01/08/1990 00:00,ND,ND,365369,865449,57.67756682,-2.58234276,...,S12000034,NaN,NaN,S11000001,S22000081,6,7,Y,1,NaN
2,AB452NP,AB45,AB45 2,01/08/1990 00:00,ND,ND,365475,865493,57.67797019,-2.58057188,...,S12000034,NaN,NaN,S11000001,S22000081,6,7,Y,1,NaN
3,AB452NW,AB45,AB45 2,12/07/1996 00:00,ND,ND,365743,865547,57.67847575,-2.57608623,...,S12000034,NaN,NaN,S11000001,S22000081,6,7,Y,1,NaN
4,AB452PJ,AB45,AB45 2,01/08/1990 00:00,ND,ND,358760,866116,57.68299892,-2.69326179,...,S12000034,NaN,NaN,S11000001,S22000081,6,7,Y,1,NaN


In [15]:
a=list(o_postcode.columns) 
a[47]

'NationalPark2010Code'

In [16]:
o_house_age=pd.read_csv("Other - SAA_PropertyAgeData.csv",encoding = 'unicode_escape', dtype='unicode')  # 'Age_Year'的dtype有问题
o_house_age.head()

,UPRN,Postcode,Building_Type,Age_Year,Age_Category,XCOORD,YCOORD
0,116000269,TD1 3SG,Detached,2002,NaN,351620,634825
1,116000270,TD1 3SG,Detached,2003,NaN,351612,634831
2,116000271,TD1 3SG,Detached,2003,NaN,351599,634836
3,116000272,TD1 3SG,Detached,2002,NaN,351588,634843
4,116000273,TD1 3SG,Detached,2003,NaN,351582,634851
